[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ClimateMatchAcademy/course-content/blob/main/projects/project-notebooks/ENSO_impact_on_precipitation_and_temperature.ipynb) &nbsp; <a href="https://kaggle.com/kernels/welcome?src=https://raw.githubusercontent.com/ClimateMatchAcademy/course-content/main/projects/project-notebooks/ENSO_impact_on_precipitation_and_temperature.ipynb" target="_parent"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open in Kaggle"/></a>

# **The Impact of ENSO on Precipitation and Temperature**

**Content creators:** Olawale Ikuyajolu & Patrick Orenstein

**Content reviewers:** Marguerite Brown, Yuxin Zhou

**Content editors:** Zane Mitrevica, Natalie Steinemann, Jenna Pearson, Chi Zhang, Ohad Zivan

**Production editors:** Wesley Banfield, Jenna Pearson, Chi Zhang, Ohad Zivan

**Our 2023 Sponsors:** NASA TOPS, Google DeepMind, and CMIP

**In this project**  you will work with climate model output, reanalysis data, and Niño 3.4 indices from CMIP5/6, ERA5, NOAA, and HadISST to understand the historical and future impacts of El Niño Southern Oscillation (ENSO) events on rainfall and temperature. You will focus on variables like sea surface temperature, surface air temperature, and precipitation. You will also be able to investigate the relationships between these variables and how they affect community efforts to prepare for the impacts of El Niño phases.

Recall from W1D1 that ENSO is a climate phenomena that originates in the tropical Pacific ocean but has global impacts on atmospheric circulation, temperature and precipitation. The two phases of ENSO are El Niño (warmer than average SSTs in the central and eastern tropical Pacific Ocean) and La Niña (cooler than average SSTs in the central and eastern tropical Pacific Ocean). The Niño 3.4 region is an area in the central and eastern Pacific Ocean that is often used for determining the phase of ENSO.

You may also reference W1D5, W2D1, and W2D4 tutorials on CMIP6 and read more about the different CMIP6 scenarios [here](https://www.dkrz.de/en/communication/climate-simulations/cmip6-en/the-ssp-scenarios). Please see the Resources section at the bottom of this notebook for more information.


# Project Template

![Project Template](https://raw.githubusercontent.com/ClimateMatchAcademy/course-content/main/projects/template-images/ENSO_template_map.svg)

*Note: The dashed boxes are socio-economic questions.*

# Data Exploration Notebook
## Project Setup

In [ ]:
# google colab installs

# !pip install condacolab &> /dev/null
# import condacolab
# condacolab.install()

# install all packages in one call (+ use mamba instead of conda)
# !mamba install xarray-datatree intake-esm gcsfs xmip aiohttp cartopy nc-time-axis cf_xarray xarrayutils "esmf<=8.3.1" xesmf &> /dev/null

In [ ]:
# imports

import time

tic = time.time()

import intake
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import xesmf as xe
from xmip.preprocessing import combined_preprocessing
from xarrayutils.plotting import shaded_line_plot
from datatree import DataTree
from xmip.postprocessing import _parse_metric
import cartopy.crs as ccrs
import pooch
import os
import tempfile

In [ ]:
# functions

%matplotlib inline

col = intake.open_esm_datastore(
    "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
)  # open an intake catalog containing the Pangeo CMIP cloud data


def load_cmip6(source_id, variable_id, member_id, table_id):  # load selected model
    cat = col.search(
        source_id=source_ids,
        variable_id=variable_id,
        member_id=member_id,
        table_id=table_id,
        grid_label="gn",
        experiment_id=[
            "historical",
            "ssp126",
            "ssp245",
            "ssp585",
        ],  # downloading the scenarios out of the total 5+historical
        require_all_on=["source_id"],
    )

    kwargs = dict(
        preprocess=combined_preprocessing,
        xarray_open_kwargs=dict(use_cftime=True),
        storage_options={"token": "anon"},
    )
    cat.esmcat.aggregation_control.groupby_attrs = ["source_id", "experiment_id"]
    dt = cat.to_datatree(**kwargs)

    return dt

In [ ]:
# helper functions

def pooch_load(filelocation=None,filename=None,processor=None):
    shared_location='/home/jovyan/shared/Data/Projects/ENSO' # this is different for each day
    user_temp_cache=tempfile.gettempdir()
    
    if os.path.exists(os.path.join(shared_location,filename)):
        file = os.path.join(shared_location,filename)
    else:
        file = pooch.retrieve(filelocation,known_hash=None,fname=os.path.join(user_temp_cache,filename),processor=processor)

    return file

## Dataset 1: Load CMIP6 Model of Your Choice

Following W2D1 (Week 2 Day 1) tutorial notebooks:

  * We use the *CESM2* model (*source_id*) and ensemble member *r4i1p1f1* (*member_id*) in this template, but you are free to select any model and ensemble member. Make sure the *member_id* selected is available for your model. You can learn more about the *member_id* and other CMIP6 facets through the links at the end of the [CMIP Resource Bank](https://github.com/ClimateMatchAcademy/course-content/blob/main/tutorials/CMIP/CMIP_resource_bank.md)
  
  * `load_cmip6` function load both historical and ssp585 (future: climate change)
  
To learn more about CMIP, including additional ways to access CMIP data, please see our [CMIP Resource Bank](https://github.com/ClimateMatchAcademy/course-content/blob/main/tutorials/CMIP/CMIP_resource_bank.md) and the [CMIP website](https://wcrp-cmip.org/). 




In [ ]:
# pick your model

source_ids = "CESM2"

dm_tas = load_cmip6(
    source_ids, "tas", "r4i1p1f1", "Amon"
)  # tas is atmoerhpere temprature
dm_pr = load_cmip6(source_ids, "pr", "r4i1p1f1", "Amon")  # pr is precipitation rate
dm_sst = load_cmip6(
    source_ids, "tos", "r4i1p1f1", "Omon"
)  # tos is surface ocean temprature
print(
    dm_tas.keys()
)  # an example for one of the datatrees, you can duplicate this for the other DT

In [ ]:
# load cell areas for computing ocean surface temparuters means

dt_ocean_area = load_cmip6(source_ids, "areacello", "r4i1p1f1", "Ofx")
dt_atmos_area = load_cmip6(source_ids, "areacella", "r4i1p1f1", "fx")

dt_ocean_with_area = DataTree()
dt_atmos_with_area = DataTree()

for model, subtree in dm_sst.items():
    metric_ocean = dt_ocean_area[model]["historical"].ds["areacello"]
    dt_ocean_with_area[model] = subtree.map_over_subtree(_parse_metric, metric_ocean)

for model, subtree in dm_pr.items():
    metric_atmos = dt_atmos_area[model]["historical"].ds["areacella"]
    dt_atmos_with_area[model] = subtree.map_over_subtree(_parse_metric, metric_atmos)

print(dt_ocean_with_area.keys())

## Dataset 2: Load Observations

We use the NOAA Extended Reconstructed Sea Surface Temperature (ERSST) v5 product, a widely used and trusted gridded compilation of historical data going back to 1854. Since the data is provided via an OPeNDAP server, we can load it directly without downloading anything.

For precipitation, we are using CPC Merged Analysis of Precipitation (CMAP). We can download this dataset from the NOAA PSL, Boulder, Colorado, USA website at https://psl.noaa.gov

For air temperature, we are using anomalies from NASA GISS Surface Temperature Analysis which we can also download from NOAA PSL, Boulder, Colorado, USA website at https://psl.noaa.gov

In [ ]:
# Ocean surface temprature 
filename_SST='sst.mnmean.nc'
url_SST = 'https://downloads.psl.noaa.gov/Datasets/noaa.ersst.v5/sst.mnmean.nc'

do_sst = xr.open_dataset(pooch_load(url_SST,filename_SST), drop_variables=['time_bnds'])

# Precipitation rate (notice the units in the plot below)
filename_prec_rate='precip.mon.mean.nc'
url_prec_rate='https://downloads.psl.noaa.gov/Datasets/cmap/enh/precip.mon.mean.nc'
do_pr = xr.open_dataset(pooch_load(url_prec_rate,filename_prec_rate))

# Air Temperature Anomalies
filename_tas='air.2x2.1200.mon.anom.comb.nc'
url_tas='https://downloads.psl.noaa.gov/Datasets/gistemp/combined/1200km/air.2x2.1200.mon.anom.comb.nc'
do_tas = xr.open_dataset(pooch_load(url_tas,filename_tas))



We can now visualize the content of the dataset.



In [ ]:
# code to print the shape, array names, etc of the dataset

# select just a single model and experiment
hist_precip = dm_pr["CESM2"]["historical"].ds.pr

fig, ax_july2000 = plt.subplots(
    ncols=1, nrows=1, figsize=[12, 6], subplot_kw={"projection": ccrs.Robinson()}
)

hist_precip.sel(time="2000-07").squeeze().plot(
    ax=ax_july2000,
    x="lon",
    y="lat",
    transform=ccrs.PlateCarree(),
    cmap="magma",
    robust=True,
)

ax_july2000.coastlines()
ax_july2000.set_title("July 2000")

In [ ]:
hist_sst = dm_sst["CESM2"]["historical"].ds.tos

fig, ax = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=[12, 6],
    subplot_kw={"projection": ccrs.Robinson(central_longitude=180)},
)

ax.coastlines()
ax.gridlines()
hist_sst.sel(time="2000-07").squeeze().plot(
    ax=ax,
    x="lon",
    y="lat",
    transform=ccrs.PlateCarree(),
    vmin=-2,
    vmax=30,
    cmap="magma",
    robust=True,
)

## Dataset 3: Oceanic Nino Index

There are several indices used to identify ENSO in the tropical Pacific Ocean. These indices are based on SST anomalies averaged across a given region and are used to define El Niño and La Niña events. Two indices that you will explore in this project are the Nino 3.4 Index and the Oceanic Niño Index (ONI). Both of these indices are averaged over the same region in the tropical Pacific (5N-5S, 170W-120W), but use different running means and criteria for identifying El Niño and La Niña events (i.e. for ONI, SST anomalies must exceed +/- 0.5C for at least five consecutive months to be defined as an ENSO event, whereas for Nino 3.4, SST anomalies must exceed +/- 0.4C for at least six consecutive months). You can find additional information about these indices [here](https://climatedataguide.ucar.edu/climate-data/nino-sst-indices-nino-12-3-34-4-oni-and-tni). For now, we will download the ONI data that we used in W1D3.

In [ ]:
# get El Nino data from W1D3 tutorial 7
filename_nino='t6_oceanic-nino-index.nc'
url_nino = "https://osf.io/8rwxb/download/"
oni = xr.open_dataset(pooch_load(url_nino,filename_nino))
print(oni.keys())


# Further Reading

- "El Niño & La Niña (El Niño-Southern Oscillation)", NOAA/Climate.gov (https://www.climate.gov/enso)
- "El Niño and La Niña", OCHA (https://www.unocha.org/themes/el-ni%C3%B1o/el-ni%C3%B1o-and-la-ni%C3%B1a)
- "How will climate change change El Nino and La Nina?", NOAA, 2020 (https://research.noaa.gov/2020/11/09/new-research-volume-explores-future-of-enso-under-influence-of-climate-change/)
- L'Heureux, M.,  Karamperidou, C., DiNezio, P., Karnauskas, K., Anyamba, A. 2023. "El Niño and La Niña: Local and global effects". Climate Central (https://www.climatecentral.org/climate-matters/local-and-global-effects-of-el-nino-and-la-nina-2023)
- Kelly, M. 2023. "In Years After El Niño, Global Economy Loses Trillions" (https://home.dartmouth.edu/news/2023/05/years-after-el-nino-global-economy-loses-trillions)

# **Resources**

This tutorial uses data from the simulations conducted as part of the [CMIP6](https://wcrp-cmip.org/) multi-model ensemble. 

For examples on how to access and analyze data, please visit the [Pangeo Cloud CMIP6 Gallery](https://gallery.pangeo.io/repos/pangeo-gallery/cmip6/index.html) 

For more information on what CMIP is and how to access the data, please see this [page](https://github.com/ClimateMatchAcademy/course-content/blob/main/tutorials/CMIP/CMIP_resource_bank.md).